In [25]:
# install the libraries from requirements.txt
#%pip install pandas
#%pip install numpy
#%pip install openpyxl
#%pip install pulp
# import the libraries
import pandas as pd
import numpy as np
import pulp
# 1. Read the databases
# a. Read the "Horarios.xlsx" file and select the "Horarios" sheet
Horarios = pd.read_excel("Base_Data\Horarios.xlsx", sheet_name="Horarios")
# b. Read the "Horarios.xlsx" file and select the "Espacios_Fisicos" sheet
Espacios_Fisicos = pd.read_excel("Base_Data\Horarios.xlsx", sheet_name="Espacios_Fisicos")
# c. Read the "Horarios.xlsx" file and select the "Tiempo" sheet
Tiempo = pd.read_excel("Base_Data\Horarios.xlsx", sheet_name="Tiempo")


In [26]:
# 2. Filter the data
#  a. Filter the "Horarios" database
#   1.1 by the "Proyecto Curricular"
#       column so only 'INGENIERIA ELECTRONICA', 'INGENIERIA ELECTRICA',
#       'INGENIERIA INDUSTRIAL', 'INGENIERIA DE SISTEMAS', 'INGENIERIA
#       CATASTRAL Y GEODESIA' are selected
Proyectos_Curriculares = ['INGENIERIA ELECTRONICA', 'INGENIERIA ELECTRICA', 'INGENIERIA INDUSTRIAL', 'INGENIERIA DE SISTEMAS', 'INGENIERIA CATASTRAL Y GEODESIA']
Horarios_ = Horarios[Horarios['Proyecto Curricular'].isin(Proyectos_Curriculares)]
#   1.2 by the "Sede" column so only 'CALLE 34 - CALLE 34', 'CALLE 40
#       - SABIO CALDAS', 'ASISTIDO POR TIC - ASISTIDO POR TIC', 
#       'CALLE 40 - ADMINISTRATIVO', 'CALLE 42 - CALLE 42', 
#       'POR ASIGNAR - POR ASIGNAR' are selected
Sedes = ['CALLE 34 - CALLE 34', 'CALLE 40 - SABIO CALDAS', 'ASISTIDO POR TIC - ASISTIDO POR TIC', 'CALLE 40 - ADMINISTRATIVO', 'CALLE 42 - CALLE 42', 'POR ASIGNAR - POR ASIGNAR']
Horarios_ = Horarios_[Horarios_['Sede'].isin(Sedes)].copy()
#   1.3 by the "Salon" column so only 'VIRT000000 - ASISTIDO POR TIC',
#    'PAS0000000 - POR ASIGNAR',
#    'FMCT010502 - AULA 502', 'FMCT010501 - AULA 501', 'FMCT010403 - AULA 403', 
#    'FMCT010402 - AULA 402', 'FMCT010303 - AULA 301', 'FMCT010302 - AULA 302', 
#    'FMCT010203 - AULA 203', 'FMCT010202 - AULA 202', 'FMCT010201 - AULA 201', 
#    'FMCT010101 - AULA 101', 'FITE050209 - AULA 208', 'FITE050208 - AULA 204', 
#    'FITE050205 - AULA 206', 'FITE050204 - AULA 203', 'FITE050203 - AULA 202', 
#    'FITE050202 - AULA 201', 'FITE050114 - AULA 102', 'FITE050112 - AULA 101', 
#    'FICC030305 - AULA 311', 'FICC030304 - AULA 310', 'FICC030301 - AULA 309', 
#    'FICC030206 - AULA 211', 'FICC030205 - AULA 208', 'FICC02S208 - AUDITORIO', 
#    'FICC020810 - AULA 804', 'FICC020802 - AULA 802', 'FICC020801 - AULA 801', 
#    'FICC020605 - AULA 600', 'FICC020505 - SALA 500', 'FICC020421 - AULA 401', 
#    'FICC020419 - AULA 403', 'FICC020414 - AULA 404', 'FICC020413 - AULA 405', 
#    'FICC020411 - AULA 407', 'FICC020405 - AULA 408', 'FICC020322 - AULA 301', 
#    'FICC020321 - AULA 302', 'FICC020320 - AULA 303', 'FICC020315 - AULA 304', 
#    'FICC020314 - AULA 305', 'FICC020312 - AULA 307', 'FICC020311 - AULA 308', 
#    'FICC020307 - AULA 309', 'FICC020306 - AULA 310', 'FICC020305 - AULA 311', 
#    'FICC020214 - AULA 201', 'FICC020213 - AULA 202', 'FICC020212 - AULA 203', 
#    'FICC020206 - AULA 204', 'FICC020205 - AULA 205', 'FICC020204 - AULA 206', 
#    are selected
Salones = Espacios_Fisicos['Salon'].tolist()
Salones += ['VIRT000000 - ASISTIDO POR TIC', 'PAS0000000 - POR ASIGNAR']
Horarios_ = Horarios_[Horarios_['Salon'].isin(Salones)].copy()
#   1.4 by the "Espacio Academico" take out  'TRABAJO DE GRADO I',
#    'TRABAJO DE GRADO I: METODOLOGIA DE LA INVESTIGACION',
#    'TRABAJO DE GRADO II', and 'TRABAJO DE GRADO',
#    'PRACTICA EMPRESARIAL', 'PRACTICA  EMPRESARIAL'
Materias_no = ['TRABAJO DE GRADO I', 'TRABAJO DE GRADO I: METODOLOGIA DE LA INVESTIGACION',
               'TRABAJO DE GRADO II', 'TRABAJO DE GRADO', 'PRACTICA EMPRESARIAL',
               'PRACTICA  EMPRESARIAL']
Horarios_ = Horarios_[~Horarios_['Espacio Academico'].isin(Materias_no)].copy()
Horarios_.keys()


Index(['AÃ‘O', 'PERIODO', 'FACULTAD', 'IDENTIFICACION', 'DOCENTE',
       'VINCULACION', 'Cod. Proyecto', 'Proyecto Curricular', 'Cod. Espacio',
       'Espacio Academico', 'Grupo', 'Inscritos', 'Edificio', 'Sede', 'Salon',
       'Dia', 'Hora'],
      dtype='object')

In [27]:
# 3. Create the information for Espacios Académicos (Materias)
#  a. Create a list of the unique values of the "Espacio Academico" column
#  b. Create a new column with the hours per week of the course
#  c. Create a new column with the number times the course is offered per week
Espacios_Academicos = Horarios_.groupby(['AÃ‘O', 'PERIODO', 'FACULTAD', 'IDENTIFICACION', 'DOCENTE',
       'VINCULACION', 'Cod. Proyecto', 'Proyecto Curricular', 'Cod. Espacio',
       'Espacio Academico', 'Grupo']
       ).agg({
    'Hora': 'count',
    'Dia': 'nunique',
    'Inscritos': 'min'}).reset_index()
#  d. Rename the column "Hora" to "Horas por semana" and "Dia" to "Dias por semana"
Espacios_Academicos = Espacios_Academicos.rename(columns={'Hora': 'Horas por semana', 'Dia': 'Dias por semana'})
# 4. Create the information for available times
#  a. change all 1 to the column name
for i in Tiempo.columns[1:]:
    if i[0] != 'w':
        Tiempo[i] = Tiempo[i].replace(1, i).copy()
#  b. Take from Tiempo groups of 2 columns, 
#     ['Lunes', 'wLunes'] and ['Martes', 'wMartes']...until ['Domingo', 'wDomingo']
#     and create a new DataFrame Tiempo_ with the columns appended
Time_Groups = [['Hora Inicio', 'Lunes', 'wLunes'],
               ['Hora Inicio', 'Martes', 'wMartes'],
               ['Hora Inicio', 'Miercoles', 'wMiercoles'],
               ['Hora Inicio', 'Jueves', 'wJueves'],
               ['Hora Inicio', 'Viernes', 'wViernes'],
               ['Hora Inicio', 'Sabado', 'wSabado'],
               ['Hora Inicio', 'Domingo', 'wDomingo']]
Tiempo_ = pd.DataFrame()
for i in Time_Groups:
    Tiempo_ = pd.concat([Tiempo_, Tiempo[i].rename(columns={i[1]:'Dia',i[2]:'w'})], axis=0)
#  c. Eliminate the rows with Dia = 0
Tiempo_ = Tiempo_[Tiempo_['Dia'] != 0].copy()    

In [28]:
# 5. Capacity analysis on Espacios_Fisicos and Espacios_Academicos
#  a. make a clasification analysis of the "Capacidad" column 
#     and create a new column "Clasificacion" with the results
#     where there are 7 categories
#     1. 0-15
#     2. 16-20
#     3. 21-25
#     4. 26-30
#     5. 31-35
#     6. 36-40
#     7. 41-200
Espacios_Fisicos['Clasificacion'] = pd.cut(Espacios_Fisicos['Capacidad'], bins=[0,15,20,25,30,35,40,200], labels=[1,2,3,4,5,6,7])
#  b. make a clasification analysis of the "Inscritos" column
#     and create a new column "Clasificacion" with the results
#     where there are 7 categories
#     1. 0-15
#     2. 16-20
#     3. 21-25
#     4. 26-30
#     5. 31-35
#     6. 36-40
#     7. 41-200
Espacios_Academicos['Clasificacion'] = pd.cut(Espacios_Academicos['Inscritos'], bins=[0,15,20,25,30,35,40,200], labels=[1,2,3,4,5,6,7])
#  c. get the number of hours per week needed for each clasification
#     and create a new DataFrame with the results
#     where the index is the clasification and the column is the number of hours
#     needed for each clasification
Clasificacion = Espacios_Academicos.groupby('Clasificacion').agg({'Horas por semana': 'sum'}).reset_index()
Clasificacion = Clasificacion.rename(columns={'Horas por semana': 'Horas por semana necesarias'})
#  d. get the number of hours per week that each room can offer,
#     wich is the sum of the Tiempo_ DataFrame
hours = Tiempo_['Dia'].count()
#  e. get the number of hours per week that each clasification can offer
#     wich is hours * number of rooms per clasification
#     and create a new column "Horas por semana disponibles" with the results
Clasificacion['Salones'] = Espacios_Fisicos.groupby('Clasificacion').agg({'Salon': 'count'})
Clasificacion['Horas por semana disponibles'] = Clasificacion['Salones'].apply(lambda x: hours * x)
# get the sum of Horas por semana disponibles and Horas por semana necesarias
# and create a row with the results
Clasificacion[['Horas por semana disponibles','Horas por semana necesarias']].sum()
if Clasificacion[['Horas por semana disponibles','Horas por semana necesarias']].sum()[0] >= Clasificacion[['Horas por semana disponibles','Horas por semana necesarias']].sum()[1]:
    print('There are enough rooms for the number of hours needed')
else:
    print('There are not enough rooms for the number of hours needed')
print('The number of hours needed is: ', Clasificacion[['Horas por semana disponibles','Horas por semana necesarias']].sum()[1])
print('The number of hours available is: ', Clasificacion[['Horas por semana disponibles','Horas por semana necesarias']].sum()[0])
print("The number of hours not ocupied is: ", Clasificacion[['Horas por semana disponibles','Horas por semana necesarias']].sum()[0] - Clasificacion[['Horas por semana disponibles','Horas por semana necesarias']].sum()[1])

There are not enough rooms for the number of hours needed
The number of hours needed is:  2954.0
The number of hours available is:  2664.0
The number of hours not ocupied is:  -290.0


In [29]:
# Get the information needed to assign the rooms
# 1. Create a list of Salones
Salones = Espacios_Fisicos['Salon']
# 2. Create a matrix where the rows are Salon and the columns are clasification
#    and the values are the bynarized values of the "Capacidad" column
#    where 1 is the value of the row and 0 is the rest
Clasificaciones = [1,2,3,4,5,6,7]
Clasificador = pd.get_dummies(Espacios_Fisicos['Clasificacion'])
Clasificador['Salon'] = Salones
Clasificador = Clasificador.set_index('Salon').astype(int)
# for each row if a value is 1 then every value at its left is 1
# and every value at its right is 0
for i in Clasificador.index:
    for j in range(1,Clasificador.shape[1]+1):
        if Clasificador.loc[i,j] == 1:
            Clasificador.loc[i,j:] = 0
            Clasificador.loc[i,:j] = 1
            break
# 3. Create a list of Proyectos Curriculares
Proyectos = Espacios_Academicos['Proyecto Curricular'].unique()
# 4. Create a list of Days and a list of Hours available for each day
tiempo = Tiempo_.groupby('Dia').agg({'Hora Inicio': 'count'}).reset_index()
tiempo = tiempo.rename(columns={'Hora Inicio': 'Horas'})
Dias = tiempo['Dia']
Horas = tiempo.set_index('Dia')['Horas']
# 5. Crear una matriz que muestre las necesidades de cada proyecto curricular por clasificacion
Demanda_C_I = {}
Demanda_C_J = pd.pivot_table(Espacios_Academicos, values='Horas por semana', index='Proyecto Curricular', columns='Clasificacion', aggfunc='sum')
Demanda_C_K = {}
Demanda_I_J = {}
Demanda_I_K = {}
Demanda_J_K = {}

# Falta relaciones :
#   Clasificación - Salones     _C-I *
#   Clasificaciones - Proyectos _C-J ✓
#   Clasificaciones - Día       _C-K 
#   Salones - Proyectos         _I-Jgit 
#   Salones - Días              _I-K
#   Proyectos - Día             _J-K

# Check the data types of all the variables

print("Salones: ", type(Salones))
print("Clasificaciones: ", type(Clasificaciones))
print("Clasificacion: ", type(Clasificador))
print("Proyectos: ", type(Proyectos))
print("Dias: ", type(Dias))
print("Horas: ", type(Horas))
print("Demanda: ", type(Demanda))

Salones:  <class 'pandas.core.series.Series'>
Clasificaciones:  <class 'list'>
Clasificacion:  <class 'pandas.core.frame.DataFrame'>
Proyectos:  <class 'numpy.ndarray'>
Dias:  <class 'pandas.core.series.Series'>
Horas:  <class 'pandas.core.series.Series'>
Demanda:  <class 'pandas.core.frame.DataFrame'>


In [30]:
# Linear Programming Model with PuLP
# to assign the rooms to the projects and days
# i: Salones
# c: Clasificaciones
# j: Proyectos
# k: Dias
# Variables:
# x[i,c,j,k] = 1 if the room i of clasification c is assigned to the project j on the day k
#             0 if the room i of clasification c is not assigned to the project j on the day k
# Objective Function:
#   Minimize the number of rooms assigned
# Parameters:
#   Demanda[j,c] = number of hours needed for the project j of clasification c
#   Clasificador[i,c] = 1 if a room can be assigned to a clasification
#                       0 if a room can not be assigned to a clasification
#   Horas[k] = number of hours available on the day k
# Constraints:
# 1. a room assigned to a project on a day can be assigned to at most one category
#    sum(c in Clasificaciones) x[i,c,j,k] <= 1
# 2. a room of category c can be assigned to a project on a day if the room can be assigned to a clasification
#   x[i,c,j,k] <= Clasificador[i,c]
# 3. there can only maximum 1 project assigned to a room of a category on a day
#   sum(j in Proyectos) x[i,c,j,k] <= 1
# 4. the asignation of a room of a category to a project on a day must satisfy the demand
#   sum(k in Dias) sum(i in Salones) x[i,c,j,k] * Horas[k] >= Demanda[j,c]
from pulp import *
model = LpProblem("Asignacion de salones", LpMinimize)
x = LpVariable.dicts("x", (Salones, Clasificaciones, Proyectos, Dias), cat='Binary')
model += lpSum(x[i][c][j][k] for i in Salones for c in Clasificaciones for j in Proyectos for k in Dias)
for i in Salones:
    for c in Clasificaciones:
        for j in Proyectos:
            for k in Dias:
                model += x[i][c][j][k] <= Clasificador.loc[i,c]
for i in Salones:
    for c in Clasificaciones:
        for k in Dias:
            model += lpSum(x[i][c][j][k] for j in Proyectos) <= 1
for i in Salones:
    for k in Dias:
        model += lpSum(x[i][c][j][k] for j in Proyectos  for c in Clasificaciones) <= 1
for j in Proyectos:
    for c in Clasificaciones:
        model += lpSum(x[i][c][j][k] * Horas[k] for i in Salones for k in Dias) >= Demanda_C_P.loc[j,c]
model.solve()
print("Status:", LpStatus[model.status])
print("Objective value:", value(model.objective))
# Write the results to a file
model.writeLP("AsignacionSalones.lp")
# save the results to an excel file where the columns are the days and the rows are the rooms and the values are the projects assigned
# and add a column with the value of the clasification that it took, plus of a column with the
# clasification shown in thr Espacios_Fisicos['Clasificacion]

Solucion = pd.DataFrame(index=Salones, columns=Dias)
Tipos = pd.DataFrame(index=Salones, columns=Dias)
for i in Salones:
    for k in Dias:
        for j in Proyectos:
            for c in Clasificaciones:
                if x[i][c][j][k].varValue == 1:
                    Solucion.loc[i,k] = j+' '+str(c)

Solucion.to_excel('Solucion.xlsx')
Tipos.to_excel('Tipos.xlsx')




c:\Users\Home\OneDrive - Universidad Distrital Francisco José de Caldas\Monitoria\ASNOM\.venv\Lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Status: Optimal
Objective value: 261.0


In [31]:
DemandaUsada = pd.DataFrame(index=Proyectos, columns=Clasificaciones)
for j in Proyectos:
    for c in Clasificaciones:
        DemandaUsada.loc[j,c] = lpSum(x[i][c][j][k] for i in Salones for k in Dias).value() * Horas[k].sum()
print(len(Salones)*len(Clasificaciones)*len(Proyectos)*len(Dias))
print(len(Salones)*len(Dias))
display(DemandaUsada)
display(Demanda)
# sum of DemandaUsada
print("Tiempo Usado: ", DemandaUsada.sum().sum())
print("Tiempo Demandado: ", Demanda.sum().sum())


11340
324


,1,2,3,4,5,6,7
INGENIERIA ELECTRONICA,108.0,156.0,96.0,48.0,96.0,84.0,48.0
INGENIERIA INDUSTRIAL,24.0,240.0,120.0,108.0,252.0,72.0,36.0
INGENIERIA CATASTRAL Y GEODESIA,12.0,84.0,108.0,84.0,168.0,96.0,36.0
INGENIERIA DE SISTEMAS,24.0,120.0,120.0,36.0,48.0,168.0,108.0
INGENIERIA ELECTRICA,60.0,96.0,60.0,72.0,60.0,24.0,60.0


Clasificacion,1,2,3,4,5,6,7
Proyecto Curricular,,,,,,,
INGENIERIA CATASTRAL Y GEODESIA,6,82,100,77,167,89,34
INGENIERIA DE SISTEMAS,24,110,114,28,48,166,98
INGENIERIA ELECTRICA,50,92,52,70,60,14,56
INGENIERIA ELECTRONICA,103,154,96,48,88,84,48
INGENIERIA INDUSTRIAL,19,229,109,101,241,72,25


Tiempo Usado:  3132.0
Tiempo Demandado:  2954


In [32]:
# show the results as tables of the combinations of rooms and days and the projects assigned and the clasification that it took